<a href="https://colab.research.google.com/github/tarakakiranmayi/DeepLearning/blob/main/Plant_disease_predictions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

In [19]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models


In [3]:
!pip install kaggle

In [4]:
kaggle_credentails=json.load(open('kaggle.json','r'))
os.environ['KAGGLE_USERNAME']=kaggle_credentails['username']
os.environ['KAGGLE_KEY']=kaggle_credentails['key']

In [5]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
100% 2.04G/2.04G [01:33<00:00, 23.9MB/s]
100% 2.04G/2.04G [01:33<00:00, 23.4MB/s]


In [6]:
!ls

kaggle.json  plantvillage-dataset.zip  sample_data


In [7]:
with ZipFile("plantvillage-dataset.zip",'r') as zip_ref:
  zip_ref.extractall()

In [8]:
print(len(os.listdir('plantvillage dataset/segmented')))
#this have leaf images
#we got with color

38


In [9]:
base_dir='plantvillage dataset/color'


In [10]:
img_size=224
batch_size=32

In [11]:
data_gen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2

)

In [12]:
train_gen=data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 43456 images belonging to 38 classes.


In [13]:
validation_gen=data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)


Found 10849 images belonging to 38 classes.


In [14]:
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_size,img_size,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(train_gen.num_classes,activation='softmax'))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      47,776,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 38)                  │           9,766 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,805,158 (182.36 MB)

 Trainable params: 47,805,158 (182.36 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
history=model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples,
    epochs=10,
    validation_data=validation_gen,
    validation_steps=validation_gen.samples
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1357/43456 ━━━━━━━━━━━━━━━━━━━━ 49:44 71ms/step - accuracy: 0.6042 - loss: 1.8681

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


43456/43456 ━━━━━━━━━━━━━━━━━━━━ 127s 3ms/step - accuracy: 0.7366 - loss: 0.9934 - val_accuracy: 0.8503 - val_loss: 0.4681
Epoch 2/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 122s 2ms/step - accuracy: 0.9146 - loss: 0.2638 - val_accuracy: 0.8728 - val_loss: 0.4182
Epoch 3/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 93s 2ms/step - accuracy: 0.9649 - loss: 0.1104 - val_accuracy: 0.8844 - val_loss: 0.4359
Epoch 4/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 103s 2ms/step - accuracy: 0.9748 - loss: 0.0755 - val_accuracy: 0.8569 - val_loss: 0.6248
Epoch 5/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 139s 2ms/step - accuracy: 0.9774 - loss: 0.0687 - val_accuracy: 0.8736 - val_loss: 0.5978
Epoch 6/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 108s 2ms/step - accuracy: 0.9839 - loss: 0.0512 - val_accuracy: 0.8795 - val_loss: 0.5720
Epoch 7/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 111s 3ms/step - accuracy: 0.9869 - loss: 0.0434 - val_accuracy: 0.8572 - val_loss: 0.8295
Epoch 8/10
43456/43456 ━━━━━━━━━━━━━━━━━━━━ 113s 3ms/step - accuracy: 0.98

In [29]:
def load_image(img_path,target_size=(224,224)):
  img=Image.open(img_path)
  img=img.resize((img_size,img_size))
  img_array=np.array(img)
  img_array=np.expand_dims(img_array,axis=0)
  img_array=img_array.astype('float32')/255.0

  return img_array
def predict_image_class(model,image_path,class_indices):
  img_array=load_image(image_path)
  predictions=model.predict(img_array)
  predicted_class_index=np.argmax(predictions[0])
  predicted_class_name=class_indices[predicted_class_index]
  return predicted_class_name


In [21]:
class_indices={v:k for k,v in train_gen.class_indices.items()}


In [25]:
image_path='/content/plantvillage dataset/color/Apple___Black_rot/0090d05d-d797-4c99-abd4-3b9cb323a5fd___JR_FrgE.S 8727.JPG'

In [30]:
predicted_class_name=predict_image_class(model,image_path,class_indices)
print("predicted class:",predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
predicted class: Apple___Black_rot
